In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib as plt
from torch.utils.data import DataLoader
from torchvision import transforms
import os
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# Fetch the current directory path
dir_path = os.getcwd()

# Append the "flowers" folder to the directory path
flowers_path = os.path.join(dir_path, "flowers")
# Fetch the flowers from the relevant subfolder
flowers = os.listdir(flowers_path)


# Create a dictionary to store the flowers and their corresponding paths
flower_paths = {}

# Loop through the flowers
for flower in flowers:
    # Fetch the path of the flower
    flower_path = os.path.join(flowers_path, flower)
    # Fetch the images of the flower
    flower_images = os.listdir(flower_path)
    # Loop through the images
    for image in flower_images:
        # Fetch the path of the image
        image_path = os.path.join(flower_path, image)
        # Store the image path in the dictionary
        flower_paths[image_path] = flower
        
        


In [3]:
# Now we have a dictionary with the image paths as keys and the flower names as values
# We will convert into dataframe with keys as one column and values as another column
flower_paths_df = pd.DataFrame.from_dict(flower_paths, orient='index')
flower_paths_df = flower_paths_df.reset_index()
flower_paths_df.columns = ['images', 'labels']

In [4]:
torch.manual_seed(32)
# Shuffle the dataframe
flower_paths_df = flower_paths_df.sample(frac=1).reset_index(drop=True)

flower_paths_df.head()

,images,labels
0,d:\BASEERX\MS - Data Science\Pytorch\flowers\s...,sunflower
1,d:\BASEERX\MS - Data Science\Pytorch\flowers\d...,dandelion
2,d:\BASEERX\MS - Data Science\Pytorch\flowers\t...,tulip
3,d:\BASEERX\MS - Data Science\Pytorch\flowers\d...,dandelion
4,d:\BASEERX\MS - Data Science\Pytorch\flowers\d...,dandelion


In [5]:
#Now we will split the dataframe into train, validation and test dataframes
X_Train,X_Test,Y_Train,Y_Test = train_test_split(flower_paths_df['images'],flower_paths_df['labels'],test_size=0.2,random_state=32)

In [6]:
from PIL import Image
#Convert the images from path to tensors
def convert_image_to_tensor(image_path):
    image = Image.open(image_path)
    image = image.resize((224,224))
    image = np.array(image)
    image = torch.from_numpy(image)
    image = image.permute(2,0,1)
    return image



#Loop through the train images and convert them to tensors

train_images = []
for image_path in X_Train:
    image = convert_image_to_tensor(image_path)
    train_images.append(image)




In [7]:
train_images = torch.stack(train_images)


In [10]:
train_images

tensor([[[[125, 131,  37,  ..., 152, 171,  20],
          [162, 155,  21,  ..., 149, 185,  18],
          [156, 145,  59,  ..., 127, 192,  29],
          ...,
          [224, 209, 193,  ..., 107,  76, 107],
          [230, 225, 217,  ...,  93,  68,  78],
          [231, 231, 227,  ...,  88,  70,  61]],

         [[ 81, 132,  37,  ..., 126, 148,  13],
          [101, 160,  35,  ..., 121, 158,   7],
          [114, 162,  73,  ...,  98, 163,  14],
          ...,
          [217, 202, 187,  ...,  94,  59,  77],
          [223, 218, 211,  ...,  71,  51,  55],
          [224, 224, 220,  ...,  58,  52,  44]],

         [[ 35,  50,   9,  ..., 110, 147,  10],
          [ 59,  80,   1,  ..., 105, 155,   4],
          [ 62,  84,  29,  ...,  80, 158,  11],
          ...,
          [201, 183, 163,  ...,  26,  11,  33],
          [207, 199, 189,  ...,  14,   5,  13],
          [207, 206, 201,  ...,  12,   4,   5]]],


        [[[218, 207, 155,  ..., 222, 183, 133],
          [211, 204, 155,  ..., 226

In [12]:
for i in Y_Train:
    if i == 'daisy':
        Y_Train.replace(i,0,inplace=True)
    elif i == 'dandelion':
        Y_Train.replace(i,1,inplace=True)
    elif i == 'rose':
        Y_Train.replace(i,2,inplace=True)
    elif i == 'sunflower':
        Y_Train.replace(i,3,inplace=True)
    elif i == 'tulip':
        Y_Train.replace(i,4,inplace=True)


In [18]:
# convert unknown column Y_Train to tensor
Y_Train = torch.from_numpy(np.array(Y_Train))

In [20]:
#Now We have X_Train, Y_Train in tensor format we will create CNN model for flowers classification

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,6,3,1)
        self.conv2 = nn.Conv2d(6,16,3,1)
        self.fc1 = nn.Linear(54*54*16,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,5)
        
    def forward(self,X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X,2,2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X,2,2)
        X = X.view(-1,54*54*16)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        return F.log_softmax(X,dim=1)

In [22]:
# Create an instance of the CNN model
model = CNN()
# Define the loss function
criterion = nn.CrossEntropyLoss()
# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the number of epochs
epochs = 10

# Define the batch size
batch_size = 10

# Define the training set
train = torch.utils.data.TensorDataset(train_images,Y_Train)

# Define the training data loader
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)

#tune the model for 100 epochs

for epoch in range(epochs):
    for data in train_loader:
        X,y = data
        model.zero_grad()
        output = model(X.float())
        loss = criterion(output,y.long())
        loss.backward()
        optimizer.step()
    print(loss)


tensor(1.3179, grad_fn=<NllLossBackward0>)
tensor(2.1169, grad_fn=<NllLossBackward0>)
tensor(0.2700, grad_fn=<NllLossBackward0>)
tensor(0.0214, grad_fn=<NllLossBackward0>)
tensor(0.0005, grad_fn=<NllLossBackward0>)
tensor(0.1486, grad_fn=<NllLossBackward0>)
tensor(0.0068, grad_fn=<NllLossBackward0>)
tensor(0.0115, grad_fn=<NllLossBackward0>)
tensor(0.0030, grad_fn=<NllLossBackward0>)
tensor(0.0014, grad_fn=<NllLossBackward0>)


In [25]:
#Converting the test images to tensors

test_images = []
for image_path in X_Test:
    image = convert_image_to_tensor(image_path)
    test_images.append(image)

test_images = torch.stack(test_images)


In [27]:
#Converting the test labels to tensors

for i in Y_Test:
    if i == 'daisy':
        Y_Test.replace(i,0,inplace=True)
    elif i == 'dandelion':
        Y_Test.replace(i,1,inplace=True)
    elif i == 'rose':
        Y_Test.replace(i,2,inplace=True)
    elif i == 'sunflower':
        Y_Test.replace(i,3,inplace=True)
    elif i == 'tulip':
        Y_Test.replace(i,4,inplace=True)
        

In [28]:
Y_Test = torch.from_numpy(np.array(Y_Test))


In [36]:
#Predicting the test images

with torch.no_grad():
    y_pred = model(test_images.float())
    y_pred = y_pred.argmax(axis=1)


#Calculating the accuracy of the model

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(Y_Test, y_pred)
print(accuracy)



0.48032407407407407


In [37]:
#Saving the model
torch.save(model.state_dict(), 'flowers_model.pth')

In [38]:
#I have a test Image
test_image = convert_image_to_tensor('test_image.jpg')



In [40]:
test_image.unsqueeze_(0)

tensor([[[[232, 232, 232,  ...,  54,  56,  55],
          [230, 230, 230,  ...,  55,  54,  52],
          [225, 225, 225,  ...,  56,  53,  51],
          ...,
          [ 17,  17,  16,  ...,  37,  35,  35],
          [ 17,  17,  15,  ...,  36,  33,  32],
          [ 17,  17,  15,  ...,  33,  31,  30]],

         [[241, 241, 241,  ...,  84,  84,  83],
          [241, 241, 241,  ...,  85,  85,  84],
          [240, 240, 240,  ...,  86,  85,  85],
          ...,
          [ 25,  25,  26,  ...,  48,  49,  50],
          [ 25,  25,  26,  ...,  45,  46,  46],
          [ 25,  25,  26,  ...,  42,  43,  44]],

         [[ 99,  99,  99,  ...,   0,   0,   0],
          [ 91,  91,  91,  ...,   1,   0,   0],
          [ 81,  80,  80,  ...,   2,   1,   1],
          ...,
          [ 12,  12,  12,  ...,  29,  27,  26],
          [ 12,  12,  12,  ...,  27,  25,  23],
          [ 12,  12,  12,  ...,  23,  22,  21]]]], dtype=torch.uint8)

In [43]:
with torch.no_grad():
    output = model(test_image.float())
    output = output.argmax(axis=1)
    print(output)

#converting the output to flower name
if output == 0:
    flower = 'daisy'
    
print(flower)

tensor([0])
daisy


In [44]:
test_image1=convert_image_to_tensor('sunflower.jpg')

In [46]:
with torch.no_grad():
    output = model(test_image1.float())
    output = output.argmax(axis=1)
    print(output)
    if output == 3:
        flower = 'sunflower'

print(flower)

tensor([3])
sunflower
